(kuberay-quickstart)=

[//]: <> (TODO: migrate this content away from ipynb)

# Getting Started

In this guide, we show you how to manage and interact with Ray clusters on Kubernetes.

You can download this guide as an executable Jupyter notebook by clicking the download button on the top right of the page.


## Preparation

### Install the latest Ray release
This step is needed to interact with remote Ray clusters using {ref}`Ray Job Submission <kuberay-job>`.

In [ ]:
! pip install -U "ray[default]"

See {ref}`installation` for more details. 

### Install kubectl

We will use kubectl to interact with Kubernetes. Find installation instructions at the [Kubernetes documentation](https://kubernetes.io/docs/tasks/tools/#kubectl).

### Access a Kubernetes cluster

We will need access to a Kubernetes cluster. There are two options:
1. Configure access to a remote Kubernetes cluster
**OR**

2. Run the examples locally by [installing kind](https://kind.sigs.k8s.io/docs/user/quick-start/#installation). Start your [kind](https://kind.sigs.k8s.io/) cluster by running the following command:

In [ ]:
! kind create cluster

To run the example in this guide, make sure your Kubernetes cluster (or local Kind cluster) can accomodate
additional resource requests of 3 CPU and 3Gi memory (for example, by setting your Docker resource limits high enough). Also, make sure your Kubernetes cluster and Kubectl are both at version at least 1.19.

## Deploying Ray cluster to Kubernetes

There are two ways to deploy the Ray cluster to Kubernetes: 
1. Deploying Ray cluster using KubeRay

2. Deploying a static Ray cluster without KubeRay

Installing CustomResourceDefinitions (CRDs) is a prerequisite to using KubeRay. Even though the KubeRay operator supports running at single namespace scope, the fact that the CRDs are cluster-scoped cannot be avoided. If the admin permission is not granted to perform the deployment, you can use the second option to deploy the Ray cluster to Kubernetes.

### Deploying Ray cluster using KubeRay

(kuberay-operator-deploy)=
#### Deploying the KubeRay operator

Deploy the KubeRay Operator by applying the relevant configuration files from the KubeRay GitHub repo.

In [ ]:
# This creates the KubeRay operator and all of the resources it needs.
! kubectl create -k "github.com/ray-project/kuberay/ray-operator/config/default?ref=v0.4.0&timeout=90s"

# Note that we must use "kubectl create" in the above command. "kubectl apply" will not work due to https://github.com/ray-project/kuberay/issues/271

# You may alternatively clone the KubeRay GitHub repo and deploy the operator's configuration from your local file system.

Confirm that the operator is running in the namespace `ray-system`.

In [ ]:
! kubectl -n ray-system get pod --selector=app.kubernetes.io/component=kuberay-operator

# NAME                                READY   STATUS    RESTARTS   AGE
# kuberay-operator-557c6c8bcd-t9zkz   1/1     Running   0          XXs

##### Namespace-scoped operator

Note that the above command deploys the operator at _Kubernetes cluster scope_; the operator will manage resources in all Kubernetes namespaces.
**If your use-case requires running the operator at single namespace scope**, refer to [the instructions at the KubeRay docs](https://ray-project.github.io/kuberay/deploy/installation/#single-namespace-version).

#### Deploying a Ray Cluster

Once the KubeRay operator is running, we are ready to deploy a Ray cluster. To do so, we create a RayCluster Custom Resource (CR).

In the rest of this guide, we will deploy resources into the default namespace. To use a non-default namespace, specify the namespace in your kubectl commands:

`kubectl -n <your-namespace> ...`

In [ ]:
# Deploy a sample Ray Cluster CR from the KubeRay repo:
! kubectl apply -f https://raw.githubusercontent.com/ray-project/kuberay/master/ray-operator/config/samples/ray-cluster.autoscaler.yaml

# This Ray cluster is named `raycluster-autoscaler` because it has optional Ray Autoscaler support enabled.

Once the RayCluster CR has been created, you can view it by running

In [ ]:
! kubectl get raycluster

# NAME                    AGE
# raycluster-autoscaler   XXs

The KubeRay operator will detect the RayCluster object. The operator will then start your Ray cluster by creating head and worker pods. To view Ray cluster's pods, run the following command:

In [ ]:
# View the pods in the Ray cluster named "raycluster-autoscaler"
! kubectl get pods --selector=ray.io/cluster=raycluster-autoscaler

# NAME                                             READY   STATUS    RESTARTS   AGE
# raycluster-autoscaler-head-xxxxx                 2/2     Running   0          XXs
# raycluster-autoscaler-worker-small-group-yyyyy   1/1     Running   0          XXs

We see a Ray head pod with two containers -- the Ray container and autoscaler sidecar. We also have a Ray worker with its single Ray container.

Wait for the pods to reach Running state. This may take a few minutes -- most of this time is spent downloading the Ray images. In a separate shell, you may wish to observe the pods' status in real-time with the following command:

In [ ]:
# If you're on MacOS, first `brew install watch`.
# Run in a separate shell:
! watch -n 1 kubectl get pod

If your pods are stuck in the Pending state, you can check for errors via `kubectl describe pod raycluster-autoscaler-xxxx-xxxxx` and ensure that your Docker resource limits are set high enough.

Note that in production scenarios, you will want to use larger Ray pods. In fact, it is advantageous to size each Ray pod to take up an entire Kubernetes node. See the [configuration guide](kuberay-config) for more details.

### Deploying a static Ray cluster


#### Deploying a static Ray cluster without KubeRay

In this section, we will deploy a static Ray cluster into the `default` namespace without using KubeRay. To use another namespace, specify the namespace in your kubectl commands:

`kubectl -n <your-namespace> ...`

In [ ]:
# Deploy a sample Ray Cluster from the Ray repo:
! kubectl apply -f https://raw.githubusercontent.com/ray-project/ray/master/doc/source/cluster/kubernetes/configs/static-ray-cluster.with-fault-tolerance.yaml

# Note that the Ray cluster has fault tolerance enabled by default using the external Redis. Please set the Redis IP address in the config.

Once the Ray cluster has been deployed, you can view the pods for the head node and worker nodes by running

In [ ]:
! kubectl get pods

# NAME                                             READY   STATUS    RESTARTS   AGE
# deployment-ray-head-xxxxx                        1/1     Running   0          XXs
# deployment-ray-worker-xxxxx                      1/1     Running   0          XXs
# deployment-ray-worker-xxxxx                      1/1     Running   0          XXs

Wait for the pods to reach the `Running` state. This may take a few minutes -- most of this time is spent downloading the Ray images. In a separate shell, you may wish to observe the pods' status in real-time with the following command:

In [ ]:
# If you're on MacOS, first `brew install watch`.
# Run in a separate shell:
! watch -n 1 kubectl get pod

If your pods are stuck in the `Pending` state, you can check for errors via `kubectl describe pod deployment-ray-head-xxxx-xxxxx` and ensure that your Docker resource limits are set high enough.

Note that in production scenarios, you will want to use larger Ray pods. In fact, it is advantageous to size each Ray pod to take up an entire Kubernetes node. See the [configuration guide](kuberay-config) for more details.

#### Deploying a network policy for the static Ray cluster


If your Kubernetes has a default deny network policy for pods, you need to manually create a network policy to allow the bidirectional communication among the head and worker nodes in the Ray cluster as mentioned in [the Ports configurations doc](https://docs.ray.io/en/latest/ray-core/configure.html#ports-configurations).

In [ ]:
# Create a sample network policy for the static Ray cluster from the Ray repo:
! kubectl apply -f https://raw.githubusercontent.com/ray-project/ray/master/doc/source/cluster/kubernetes/configs/static-ray-cluster-networkpolicy.yaml

Once the network policy has been deployed, you can view the network policy for the static Ray cluster by running

In [ ]:
! kubectl get networkpolicies

# NAME                               POD-SELECTOR                           AGE
# ray-cluster-egress                 app=ray-cluster-head                   XXs
# ray-cluster-ingress                app=ray-cluster-head                   XXs

#### External Redis Integration for fault tolerance
Ray by default uses an internal key-value store, called the Global Control Store (GCS). The GCS runs on the head node and stores cluster metadata. One drawback of this approach is that the head node loses the metadata if it crashes. 
Ray can also write this metadata to an external Redis for reliability and high availability. With this setup, the static Ray cluster can recover from head node crasehs and tolerate GCS failures without losing connections to worker nodes.

To use this feature, we need to pass in the RAY_REDIS_ADDRESS env var and --redis-password in the Ray head node section of [the Kubernetes deployment config file](https://raw.githubusercontent.com/ray-project/ray/master/doc/source/cluster/kubernetes/configs/static-ray-cluster.with-fault-tolerance.yaml).

## Running Applications on a Ray Cluster

Now, let's interact with the Ray cluster we've deployed.

### Accessing the cluster with kubectl exec

The most straightforward way to experiment with your Ray cluster is to
exec directly into the head pod. First, identify your Ray cluster's head pod:

If the Ray cluster was deployed using KubeRay, run the command below to get the head pod:

In [ ]:
! kubectl get pods --selector=ray.io/cluster=raycluster-autoscaler --selector=ray.io/node-type=head -o custom-columns=POD:metadata.name --no-headers
    
# raycluster-autoscaler-head-xxxxx

If the Ray cluster was deployed using the static Ray cluster config, run the command below to get the head pod:

In [ ]:
! kubectl get pods --selector=app=ray-cluster-head

Now, we can run a Ray program on the head pod. The Ray program in the next cell simply connects to the Ray Cluster, then exits.

In [ ]:
# Substitute your output from the last cell in place of "raycluster-autoscaler-head-xxxxx"

! kubectl exec raycluster-autoscaler-head-xxxxx -it -c ray-head -- python -c "import ray; ray.init()"
# 2022-08-10 11:23:17,093 INFO worker.py:1312 -- Connecting to existing Ray cluster at address: <IP address>:6379...
# 2022-08-10 11:23:17,097 INFO worker.py:1490 -- Connected to Ray cluster.

While this can be useful for ad-hoc execution on the Ray Cluster, the recommended way to execute an application on a Ray Cluster is to use [Ray Jobs](jobs-quickstart).

(kuberay-job)=
### Ray Job submission

To set up your Ray Cluster for Ray Jobs submission, we just need to make sure that the Ray Jobs port is visible to the client.
Ray listens for Job requests through the head pod's Dashboard server.

First, we need to find the location of the Ray head node. Both the KubeRay operator and the static Ray cluster configuration file configure a [Kubernetes service](https://kubernetes.io/docs/concepts/services-networking/service/) targeting the Ray head pod. This service lets us interact with Ray clusters without directly executing commands in the Ray container. To identify the Ray head service for our example cluster, run:

In [2]:
# If the Ray cluster is deployed using KubeRay
! kubectl get service raycluster-autoscaler-head-svc

# NAME                             TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)                       AGE
# raycluster-autoscaler-head-svc   ClusterIP   10.96.114.20   <none>        6379/TCP,8265/TCP,10001/TCP   XXs

In [ ]:
# If the Ray cluster is deployed using the static Ray cluster configuration file
! kubectl get service service-ray-cluster

# NAME                             TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)                            AGE
# service-ray-cluster              ClusterIP   10.96.114.20   <none>        6379/TCP,8265/TCP,10001/TCP...     XXs

Now that we have the name of the service, we can use port-forwarding to access the Ray Dashboard port (8265 by default).

Note: The following port-forwarding command is blocking. If you are following along from a Jupyter notebook, the command must be executed in a separate shell outside of the notebook.

In [ ]:
# Execute this in a separate shell.
# Substitute the service name in place of "raycluster-autoscaler-head-svc"
! kubectl port-forward service/raycluster-autoscaler-head-svc 8265:8265

Note: We use port-forwarding in this guide as a simple way to experiment with a Ray cluster's services. For production use-cases, you would typically either 
- Access the service from within the Kubernetes cluster or
- Use an ingress controller to expose the service outside the cluster.

See the {ref}`networking notes <kuberay-networking>` for details.

Now that we have access to the Dashboard port, we can submit jobs to the Ray Cluster:

In [ ]:
# The following job's logs will show the Ray cluster's total resource capacity, including 3 CPUs.

! ray job submit --address http://localhost:8265 -- python -c "import ray; ray.init(); print(ray.cluster_resources())"

For a more detailed guide on using Ray Jobs to run applications on a Ray Cluster, check out the [quickstart guide](jobs-quickstart)

## Cleanup

### Deleting a Ray Cluster

To delete the KubeRay Ray Cluster we deployed in this example, run the following command.

In [ ]:
# Delete by reference to the RayCluster custom resource
! kubectl delete raycluster raycluster-autoscaler

Confirm that the Ray Cluster's pods are gone by running

In [ ]:
! kubectl get pods

Note that it may take several seconds for the Ray pods to be fully terminated.

### Deleting the KubeRay operator

In typical operation, the KubeRay operator should be left as a long-running process that manages many Ray clusters.
If you would like to delete the operator and associated resources, run

In [ ]:
! kubectl delete -k "github.com/ray-project/kuberay/ray-operator/config/default?ref=v0.4.0&timeout=90s"

### Deleting the static Ray cluster
If you'd like to delete the static Ray cluster, run

In [ ]:
# Delete the static Ray cluster service and deployments
! kubectl delete -f https://raw.githubusercontent.com/ray-project/ray/master/doc/source/cluster/kubernetes/configs/static-ray-cluster.with-fault-tolerance.yaml

# Delete the static Ray cluster network policy
! kubectl delete -f https://raw.githubusercontent.com/ray-project/ray/master/doc/source/cluster/kubernetes/configs/static-ray-cluster-networkpolicy.yaml

### Deleting a local kind cluster

Finally, if you'd like to delete your local kind cluster, run

In [ ]:
! kind delete cluster